In [5]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

os_env = 'mac'

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/'

if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader_new_version0229 import *
import eicu_year_process

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/'

if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [6]:
if os_env == 'window':
    mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

else:
    mimic_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

In [7]:
mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]
# dataset_mimic = dataset_mimic[~((dataset_mimic['INDEX']=='CASE3_CASE4_DF')&(dataset_mimic['Annotation']=='no_circ'))]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
# eicu_test = eicu_test[~((eicu_test['INDEX']=='CASE3_CASE4_DF')&(eicu_test['Annotation']=='no_circ'))]
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')
hosp_id, eicu_test = get_hospital_eicu.hospital(eicu_test)

eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test)
unitadmitsource, unittype, unitstaytype = get_hospital_eicu.make_eicu_dataset(eicu_type)

Trial:  0
Threshold 조정 + 0.05, 현재 한계값: 0.1
========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    174346
2.0     27608
3.0     73356
4.0     71699
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    17664
2.0     3030
3.0     9822
4.0     8278
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.50:0.08:0.21:0.21
테스트셋 클래스 비율: 0.46:0.08:0.25:0.21
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 5323
테스트셋 환자 수: 601
학습셋 체류 수: 5433
테스트셋 체류 수: 604
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.1
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.30
시도한 시행 횟수: 0


In [134]:
if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/incorperate_subtask_label.py'>

In [135]:
mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [136]:
los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [137]:
ards4h_mimic_train, ards4h_mimic_valid, ards4h_eicu_test = incorperate_subtask_label.ARDS4_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [138]:
ARDS4h_event_train = ards4h_mimic_train[ards4h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS4h_event_train['ARDS_next_4h']='event'
ARDS4h_event_valid = ards4h_mimic_valid[ards4h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS4h_event_valid['ARDS_next_4h']='event'
ARDS4h_event_test = ards4h_eicu_test[ards4h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS4h_event_test['ARDS_next_4h']='event'

In [139]:
ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [140]:
ARDS8h_event_train = ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'
ARDS8h_event_valid = ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS8h_event_valid['ARDS_next_8h']='event'
ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'

In [141]:
sic4h_mimic_train, sic4h_mimic_valid, sic4h_eicu_test = incorperate_subtask_label.SIC4_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [142]:
SIC4h_event_train = sic4h_mimic_train[sic4h_mimic_train['Annotation_SIC']=='SIC']
SIC4h_event_train['SIC_next_4h']='event'
SIC4h_event_valid = sic4h_mimic_valid[sic4h_mimic_valid['Annotation_SIC']=='SIC']
SIC4h_event_valid['SIC_next_4h']='event'
SIC4h_event_test = sic4h_eicu_test[sic4h_eicu_test['Annotation_SIC']=='SIC']
SIC4h_event_test['SIC_next_4h']='event'

In [143]:
sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [144]:
SIC8h_event_train = sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'
SIC8h_event_valid = sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='SIC']
SIC8h_event_valid['SIC_next_8h']='event'
SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [157]:
reload(split)

<module 'split' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/split.py'>

In [158]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mort_mimic_train[~(mort_mimic_train['death']=='event')], mode = 'mimic')
X_valid_mort, y_valid_mort, valid_output_mort = split.split_X_Y_MORT(mort_mimic_valid[~(mort_mimic_valid['death']=='event')], mode = 'mimic')

X_train_los, y_train_los, _ = split.split_X_Y_LOS(los_mimic_train, mode = 'mimic')
X_valid_los, y_valid_los, valid_output_los = split.split_X_Y_LOS(los_mimic_valid, mode = 'mimic')

X_train_ards4h, y_train_ards4h, _ = split.split_X_Y_ARDS4h(ards4h_mimic_train[ards4h_mimic_train['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')
X_valid_ards4h, y_valid_ards4h, valid_output_ards4h = split.split_X_Y_ARDS4h(ards4h_mimic_valid[ards4h_mimic_valid['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')

X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')
X_valid_ards8h, y_valid_ards8h, valid_output_ards8h = split.split_X_Y_ARDS8h(ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')

X_train_sic4h, y_train_sic4h, _ = split.split_X_Y_SIC4h(sic4h_mimic_train[sic4h_mimic_train['Annotation_SIC']=='no_SIC'], mode = 'mimic')
X_valid_sic4h, y_valid_sic4h, valid_output_sic4h = split.split_X_Y_SIC4h(sic4h_mimic_valid[sic4h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='no_SIC'], mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [164]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [165]:
mort_models, result_mort = Multiclass_evaluation.create_subtask(X_train_mort, y_train_mort, X_valid_mort, valid_output_mort, _,mode = 'mimic', type = 'binary', event_task = False)

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,F1 Score,Recall,Precision
0,xgb,0.669375,0.520665,0.470217,0.462374,0.478331
1,lgbm,0.667159,0.498034,0.451744,0.432118,0.473237
2,catboost,0.690300,0.524256,0.495994,0.480217,0.512842
3,rf,0.685623,0.470319,0.331063,0.245151,0.509677
4,dt,0.559330,0.332412,0.396086,0.455392,0.350448
5,svm-ovr,0.680207,0.399480,0.081979,0.044996,0.460317
6,lr,0.656819,0.402474,0.272443,0.202483,0.416268
7,naivebayes,0.317331,0.403747,0.481779,1.000000,0.317331
8,knn,0.590103,0.321307,0.325091,0.311094,0.340407


----------------------------------------------------------------------


In [129]:
result_mort.to_excel(excel_writer='mort_baseline_method.xlsx')

In [166]:
los_models, result_los = Multiclass_evaluation.create_subtask(X_train_los, y_train_los, X_valid_los, valid_output_los, _, mode = 'mimic', type = 'multi', event_task = False)

In [128]:
result_los.to_excel(excel_writer='los_baseline_method.xlsx')

In [ ]:
ards4h_models, result_ards4h = Multiclass_evaluation.create_subtask(X_train_ards4h, y_train_ards4h, X_valid_ards4h, valid_output_ards4h, ARDS4h_event_valid, mode = 'mimic', type = 'binary', event_task='ARDS 4h')

In [248]:
ards8h_models, result_ards8h = Multiclass_evaluation.create_subtask(X_train_ards8h, y_train_ards8h, X_valid_ards8h, valid_output_ards8h, ARDS8h_event_valid, mode = 'mimic', type = 'binary', event_task='ARDS 8h')

|MIMIC-Validation|====================================================


,Model,AUPRC,ARDS score,Recall,Precision,Grecall
0,lgbm,0.382292,0.492114,0.6667,0.375000,0.571429
1,rf,0.490242,0.633238,0.8333,0.425000,0.809524
2,dt,0.678448,0.655786,1.0000,0.459459,0.809524
3,lr,0.631152,0.750704,1.0000,0.493976,0.976190


----------------------------------------------------------------------


In [250]:
result_ards8h.to_excel(excel_writer='ards8h_baseline_method.xlsx')

In [251]:
X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train, mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [252]:
sic8h_models, result_sic8h = Multiclass_evaluation.create_subtask(X_train_sic8h, y_train_sic8h, X_valid_sic8h, valid_output_sic8h, SIC8h_event_valid, mode = 'mimic', type = 'binary', event_task='SIC 8h')

|MIMIC-Validation|====================================================


,Model,AUPRC,SIC score,Recall,Precision,Grecall
0,lgbm,0.368339,0.240370,0.222222,0.279070,0.226415
1,rf,0.416353,0.234811,0.333333,0.333333,0.207547
2,dt,0.580979,0.415525,0.444444,0.466667,0.396226
3,lr,0.370824,0.487710,0.555556,0.391892,0.547170


----------------------------------------------------------------------


In [253]:
result_sic8h.to_excel(excel_writer='sic8h_baseline_method.xlsx')